# LLMCheckerChain
This notebook showcases how to use LLMCheckerChain.

In [1]:
from langchain.chains import LLMCheckerChain, LLMChain, SequentialChain, ParallelChain
from langchain.llms import OpenAI
from langchain.prompts.base import ListOutputParser
from langchain.llms.base import LLM


llm = OpenAI(temperature=0.7)

text = "What type of mammal lays the biggest eggs?"

checker_chain = LLMCheckerChain(llm=llm, verbose=True)

# checker_chain.run(text)

In [2]:
# flake8: noqa
from langchain.prompts.prompt import PromptTemplate

_CREATE_DRAFT_ANSWER_TEMPLATE = """{question}\n\n"""
CREATE_DRAFT_ANSWER_PROMPT = PromptTemplate(
    input_variables=["question"], template=_CREATE_DRAFT_ANSWER_TEMPLATE
)

_LIST_ASSERTIONS_TEMPLATE = """Here is a statement:
{statement}
Make a bullet point list of the assumptions you made when producing the above statement.\n\n"""
LIST_ASSERTIONS_PROMPT = PromptTemplate(
    input_variables=["statement"], template=_LIST_ASSERTIONS_TEMPLATE
)

_CHECK_ASSERTIONS_TEMPLATE = """{assertions}
Repeat the assertion above, and then determine if it is true or false. If it is false, explain why.\n\n"""
CHECK_ASSERTIONS_PROMPT = PromptTemplate(
    input_variables=["assertions"], template=_CHECK_ASSERTIONS_TEMPLATE
)

_REVISED_ANSWER_TEMPLATE = """{checked_assertions}

Question: In light of the above assertions and checks, how would you answer the question '{question}'?

Answer:"""
REVISED_ANSWER_PROMPT = PromptTemplate(
    input_variables=["checked_assertions", "question"],
    template=_REVISED_ANSWER_TEMPLATE,
)

In [3]:
llm = OpenAI(temperature=0.7)

create_draft_answer_chain = ParallelChain(
    llm=llm, prompt=CREATE_DRAFT_ANSWER_PROMPT, output_key="statement"
)

list_assertions_chain = ParallelChain(
    llm=llm,
    prompt=LIST_ASSERTIONS_PROMPT,
    output_key="assertions",
    # This indicates that there are multiple outputs
    output_parser=lambda x: [i.strip() for i in x.split("\n") if len(i.strip()) > 0],
)

check_assertions_chain = ParallelChain(
    llm=llm, prompt=CHECK_ASSERTIONS_PROMPT, output_key="checked_assertions"
)

revised_answer_chain = ParallelChain(
    llm=llm,
    prompt=REVISED_ANSWER_PROMPT,
    output_key="revised_answer",
    # This indicates that there are multiple inputs
    input_parser=lambda x: "\n".join(x),
)

self_ask_verification_chain = SequentialChain(
    chains=[
        create_draft_answer_chain,
        list_assertions_chain,
        check_assertions_chain,
        revised_answer_chain,
    ],
    input_variables=["question"],
    output_variables=["revised_answer"],
    verbose=True,
)

In [4]:
%%time
self_ask_verification_chain.run(text)



> Entering new SequentialChain chain...
Chain 0:
{'statement': ['\nThe platypus lays the largest eggs relative to its body size of any mammal.']}

Chain 1:
{'assertions': ['• The platypus is a mammal', '• The platypus lays eggs', '• The platypus lays the largest eggs relative to its body size compared to other mammals', '• The size of the platypus eggs can be compared with other mammals']}

Chain 2:
{'checked_assertions': ['The platypus is a mammal.\nTrue. The platypus is a mammal because it is a warm-blooded, egg-laying animal with fur and mammary glands that produce milk to feed its young.', 'The platypus lays eggs is true. Platypuses are monotremes, which means they lay eggs rather than giving birth to live young like most other mammals.', 'The platypus lays the largest eggs relative to its body size compared to other mammals. FALSE. While the platypus does lay large eggs relative to its body size, other mammals such as the echidna lay larger eggs relative to its body size.', '\nT

[' The echidna lays the biggest eggs relative to its body size compared to other mammals.']

In [5]:
llm = OpenAI(temperature=0.7)

create_draft_answer_chain = ParallelChain(
    llm=llm, prompt=CREATE_DRAFT_ANSWER_PROMPT, output_key="statement", n = 25
)

list_assertions_chain = ParallelChain(
    llm=llm,
    prompt=LIST_ASSERTIONS_PROMPT,
    output_key="assertions",
    # This indicates that there are multiple outputs
#     output_parser=lambda x: [i.strip() for i in x.split("\n") if len(i.strip()) > 0],
)

check_assertions_chain = ParallelChain(
    llm=llm, prompt=CHECK_ASSERTIONS_PROMPT, output_key="checked_assertions"
)

revised_answer_chain = ParallelChain(
    llm=llm,
    prompt=REVISED_ANSWER_PROMPT,
    output_key="revised_answer",
    # This indicates that there are multiple inputs
#     input_parser=lambda x: "\n".join(x),
)


MOE_CONSENSUS_PROMPT = PromptTemplate(
    input_variables=["revised_answer", "question"],
    template="""{revised_answer}

Question: A group of experts each gave their opinion above. How would the plurality of them answer the question '{question}'?

Answer:""",
)

moe_answer_chain = ParallelChain(
    llm=llm,
    prompt=MOE_CONSENSUS_PROMPT,
    output_key="final_answer",
    # This indicates that there are multiple inputs
    input_parser=lambda x: "\n -" + "\n -".join(x),
)


self_ask_verification_chain = SequentialChain(
    chains=[
        create_draft_answer_chain,
        list_assertions_chain,
        check_assertions_chain,
        revised_answer_chain,
        moe_answer_chain,
    ],
    input_variables=["question"],
    output_variables=["final_answer"],
    verbose=True,
)

In [6]:
%%time
self_ask_verification_chain.run(text)



> Entering new SequentialChain chain...
Chain 0:
{'statement': ['\nThe largest mammal egg is laid by the elephant shrew, a small mammal native to Africa.', '\nThe largest mammal that lays eggs is the Australian spiny anteater, also known as the echidna.', '\n\nThe largest egg laid by a mammal belongs to the whale shark, a species of fish.', '\nThe largest mammal that lays eggs is the platypus.', '\nThe largest mammal that lays eggs is the elephant bird, an extinct species from Madagascar.', '\nThe largest eggs laid by a mammal are those of the Australian spiny anteater, or echidna. These eggs can be up to 10 times larger than a chicken egg.', '\nThe largest mammal to lay eggs is the duck-billed platypus.', '\nThe largest egg laid by a mammal belongs to the North Pacific right whale, which can lay eggs measuring up to 20 inches (50 cm) long and 12 inches (30 cm) wide.', '\nThe biggest mammal egg is laid by the elephant bird, an extinct species native to Madagascar.', '\n\nThe biggest 

Chain 3:
{'revised_answer': [' Elephant shrews lay the biggest mammal eggs.', ' The Australian spiny anteater, also known as an echidna, is the largest mammal that lays eggs.', ' The largest mammal eggs are laid by the elephant bird, which is an extinct species of giant flightless bird found on the island of Madagascar.', ' The echidna, also known as the spiny anteater, lays the biggest eggs among mammals.', ' No mammal lays eggs, so there is no answer to this question.', ' The Australian spiny anteater/echidna lays the largest eggs of any mammal.', ' The duck-billed platypus is the only mammal that lays eggs, making it the largest mammal to lay eggs.', ' The North Pacific right whale lays the biggest eggs of any mammal, reaching up to 20 inches in length and 12 inches in width.', ' The elephant bird, an extinct species native to Madagascar, lays the biggest mammal egg.', ' North American beavers lay the biggest eggs of any mammal, with eggs up to 7.5 cm (3 in) long.', ' The echidna is

[' The echidna is the largest mammal that lays eggs.']